In [66]:
from pathlib import Path
from typing import Any, Callable
from collections import namedtuple
from openai import OpenAI
from dotenv import load_dotenv
from collections import defaultdict
import json, os, csv

load_dotenv()


True

## news_id 매핑

In [79]:
id_map = {'N-1-1300101-20250512': 1, 
        'N-5-1100901-20250512': 2, 
        'N-4-1300201-20250512': 3, 
        'error': 4, 
        'N-2-1100201-20250512': 5, 
        'N-1-8100401-202505111': 6, 
        'N-5-4104008-202505112': 7,
        'N-4-1200201-202505111': 8, 
        'N-3-1500601-202505111': 9,
        'N-2-1501201-202505111': 10,
        'N-1-8100401-20250512': 11,
        'N-5-1500401-202505122': 12, 
        'N-4-1400351-202505122': 13, 
        'N-3-1300201-20250512': 14, 
        'N-2-1300201-20250512': 15, 
        'N-1-8100401-20250511': 16, 
        'N-5-1400351-20250511': 17, 
        'N-4-2100101-20250511': 18, 
        'N-3-8100401-20250511': 19, 
        'N-2-4101008-20250511': 20, 
        'error_02': 21, 
        'N-6-1100611-202505121': 22,
        'N-5-1400351-202505122': 23,
        'N-4-1100901-20250512': 24, 
        'N-3-1100801-20250512': 25,
        'N-2-2100311-20250512': 26
}

## statics.txt 생성

In [81]:
loc_news_id = {
    "01_News": 0,
    "02_Domain" : 2,
    "03_Word": 5,
    "04_PNComment": 3,
    "05_Summary": 3
}



data_type = ["01_News", "02_Domain", "03_Word", "04_PNComment", "05_Summary"]
base_path = Path("/Users/minji/Desktop/Test/NewsSummarization/data/SQL")
statistics_save_path = Path("/Users/minji/Desktop/Test/NewsSummarization/data/SQL/statistics.txt")

for news_id in range(1,27):
    data_statistics = {
        "01_News":0,
        "02_Domain":0,
        "03_Word":0,
        "04_PNComment": 0,
        "05_Summary":0
    }
    
    for type in data_type:
        #데이터 타입별 경로
        data_path = base_path / type
        
        #도메인별 경로 접근
        for domain in data_path.glob("*.txt"):
            #동일 도메인별 집합
            with open(domain, "r", encoding="utf-8") as f:
                data = f.readlines()
                
            for record in data:
                try:
                    record = eval(record.strip(",\n"))
                except Exception as e: print(record)
                
                
                if news_id == record[loc_news_id[type]] : data_statistics[type] += 1
            # loc_news_id[domain.name] : 아이디 위치
            
    with open(statistics_save_path, "a", encoding="utf-8") as f:
        print(f"news: No.{news_id}",file=f)
        for type, count in data_statistics.items():
            print(type, count, file=f)
        print("-"*10, file=f)

# news_id 포맷 변경

## news_id 변경

In [80]:
target = {"01_News":0, "02_Domain":2, "03_Word":4, "04_PNComment":3, "05_Summary":3}

for folder in target:
    if folder != "01_News" : continue
    SQL_format = Path(f"/Users/minji/Desktop/Test/NewsSummarization/data/SQL/{folder}")
    
    for file in SQL_format.glob("*.txt"):
        new_table = []
        
        with open(file, "r", encoding="utf-8") as f:
            table = f.readlines()
        
        for record in table:
            record = list(eval(record.strip(",\n")))
            #뉴스 아이디 위치
            record[target[folder]] = id_map[record[target[folder]]]
            new_table.append(str(tuple(record)))
        
        with open(file, "w", encoding="utf-8") as f:
            f.write(",\n".join(new_table))
            
      

# OutputJson_summary id 변환

In [41]:
#data/ generate_data에서 반복 실행
#경제 1 -
domains = "경제,국제,사회,정치,IT_과학".split(",")
num = 1
for domain in domains:
    summary_data = Path(f"/Users/minji/Desktop/Test/NewsSummarization/data/generate_data/{domain}/Summary")
    
    for summary in summary_data.glob("*.json"):
        with open(summary, 'r', encoding="utf-8") as f:
            json_data = json.load(f)
        
        split_id = json_data["summary_id"].split("-")
        split_id[-1] = str(num) ; num+=1
        json_data["summary_id"] = "".join(split_id)
        
        with open(summary, "w", encoding="utf-8") as f:
            json.dump(json_data, f, ensure_ascii=False, indent=4)
    

# SQL 형식 변환

In [144]:
#뉴스데이터 형식 변환
NewsData = namedtuple("NewsData", ["news_id", "uuid", "title", "author", "content", "views", "thumbnail_url", "created_at"])

#요약데이터 형식 변환
Summary = namedtuple("Summary", ["summary_id", "level", "summary_content", "news_id"] )

#코멘트 데이터 형식 변환
PNComment = namedtuple("PNComment", ["pn_evaluation_id", "positive_comment", "negative_comment", "news_id"])

#도메인 데이터 형식 변환
Domain = namedtuple("Domain", ["domain_id","domain","news_id"])

#단어 형식 변환
Word = namedtuple("Word", ["word_id","word_definition","sentence", "level", "news_id"])

## csv로 파일 변환

In [ ]:
base = Path("/Users/minji/Desktop/Test/NewsSummarization/data/SQL/01_News")

for domain in base.glob("*.txt"):
    with open(domain, "r", encoding="utf-8") as f:
        data = f.readlines()
        
    with open(domain.with_suffix(".csv"), "w", newline="", encoding="utf-8") as fout:
        writer = csv.writer(fout, quoting=csv.QUOTE_ALL)
        
        for row in data:
            record = list(eval(row.rstrip(",\n")))

            # 문자열 필드 안의 실제 줄바꿈을 다시 '\\n' 문자열로 되돌림
            record = [
                field.replace("\n", "\\n") if isinstance(field, str) else field
                for field in record
            ]

            writer.writerow(record)
            
        
    

## 명령문 파일

In [ ]:
def escape_sql(s: str) -> str:
    return s.replace("'","''").replace("\n","\\n")

base = Path("/Users/minji/Desktop/Test/NewsSummarization/data/SQL/01_News/csv")

for domain in base.glob("*.csv"):
    with open(domain, "r", encoding="utf-8") as fin, open(domain.with_suffix(".sql"), "w", encoding="utf-8" ) as fout:
        reader = csv.reader(fin)
        
        #한 행씩 리딩
        for row in reader:
            sql_val = []
            
            for val in row:
                if val.lower() == "nan":
                    sql_val.append("NULL")
                elif val.isnumeric() : sql_val.append(val)
                else:
                    safe = escape_sql(val)
                    sql_val.append(f"'{safe}'") #이렇게 해야 insert문에 '' 잘 표기 되는 군... " "는 파이썬에서 문자열을 작성할때나 그럴 때 필수로 써야하는거고 출력할땐 빼고 나오잖니, 그런 이치 아닐가>
            insert_format = f"INSERT INTO News VALUES ({', '.join(sql_val)});"
            fout.write(insert_format + "\n")

In [ ]:

fileds = {
    "Domain": ("id", "title", "content"),
    "Word" : ("word_id","word_definition","sentence", "level", "news_id"),
    "PNComment":("pn_evaluation_id", "positive_comment", "negative_comment", "news_id"),
    "Summary": ("summary_id", "level", "summary_content", "news_id"),
        
}






base_path = Path("/Users/minji/Desktop/Test/NewsSummarization/data/SQL")

#0으로 시작하는 폴더 찾기
for type in base_path.glob("0*"):
    type_name = type.name.split("_")[1]
    
    for domain in type.glob("*.txt"):
        with open(domain, "r", encoding="utf-8") as fin:
            data = f.readlines(f)
            table = []
            
            for record in data:
                #python 객체로 변환
                record = tuple(eval(record.rstrip(",\n")))
                table.append(record)
                
            insert_format = f"INSERT INTO {type_name} {fileds[type_name]}VALUES{",\n".join(table)}"

In [77]:
import re

def convert_to_double(field: str) -> str:
    #따옴표 제외 컨텐츠 섹션만 뽑아옴
    match = re.match(r"^'(.*)'$", field)
    if match:
        content = match.group(1).replace('"','""') #내부 따옴표 처리 ""(2번 적어야 함)
        return content
    return field

In [122]:
base = Path("/Users/minji/Desktop/Test/NewsSummarization/data/SQL/01_News")

for csv_file in base.glob("*.csv"):
    with open(csv_file, "r", encoding="utf-8") as fin:
        data = csv.reader(fin)
        #한 행씩 가져옴   
        converted = []
        for row in data:
            converted = [convert_to_double(field) for field in row]
        
    with open(csv_file, "w", encoding="utf-8", newline="") as fout:
        writer = csv.writer(fout)
        writer.writerow(converted)
    
        
        

In [147]:
def trasfer_news(root: Path):
    for domain in root.iterdir():
        table = []
        if domain.name == "process_data.ipynb": continue
    
        for data in domain.glob("*.json"):
            with open(data, "r", encoding="utf-8") as f:
                json_data = json.load(f)
                
            sql_obj = NewsData(
                news_id = json_data["news_id"],
                uuid = None,
                title=json_data["title"],
                author=json_data["author"],
                content=json_data["content"],
                views=0,
                thumbnail_url=json_data["imageUrl"],
                created_at=json_data["creatAt"][:4]+"-"+ json_data["creatAt"][4:6]+"-"+json_data["creatAt"][6:]
            )
            
            table.append(str(tuple(sql_obj)))
        with open(domain.parent / f"{domain.name}.txt", "w", encoding="utf-8") as f:
            f.write(",\n".join(table))
    
    return


def transfer_word(root: Path):
    table = []
    for domain in root.iterdir():
        if domain.name.startswith("."): continue
        words = Path(f"/Users/minji/Desktop/Test/NewsSummarization/data/generate_data/{domain.name}/Word")
        
        for file in words.iterdir():
            if file.name.startswith(".") : continue
            
            with open(file, "r", encoding="utf-8") as f:
                json_data = json.load(f)
            
            for word in json_data:
                sql_obj = Word(
                    word_id= word["word"],
                    word_definition=word["definition"],
                    sentence=word["sentence"],
                    level=word["level"],
                    news_id=word["newsId"]
                )
                
                table.append(str(tuple(sql_obj)))      
        save_path = Path(f"/Users/minji/Desktop/Test/NewsSummarization/data/SQL/03_Word/{domain.name}.txt")
        with open(save_path, "w", encoding="utf-8") as f:
            f.write(",\n".join(table))
    
    
    return

def transfer_domain():
    domains = "경제,국제,사회,정치,IT_과학".split(",") ; cnt = 1
    #도메인 id 매핑정보
    domain_id = {
        "경제": "1",
        "국제" : "2",
        "사회" : "3",
        "정치" : "4",
        "IT_과학" : "5"
    }
    
    save_root = Path("/Users/minji/Desktop/Test/NewsSummarization/data/SQL/02_Domain")
    
    for domain in domains:
        table = []
        with open(Path(f"/Users/minji/Desktop/Test/NewsSummarization/data/SQL/01_News/{domain}.txt"), "r", encoding="utf-8") as f:
            data = f.readlines()
        
        for news in data:
            news_tuple = eval(news.strip(",\n"))
            id = news_tuple[0] #뉴스 id 추출
        
            save_format = (domain_id[domain], domain, id)
            table.append(str(save_format))

        with open(save_root / f"{domain}.txt" , "w", encoding="utf-8") as f:
            f.write(",\n".join(table))
    
    return
    
def transfer_pn_evaluation():
    for domain in Path("/Users/minji/Desktop/Test/NewsSummarization/data/generate_data").iterdir():
        table = []
        if domain.name.startswith(".") : continue
        comments = Path(f"/Users/minji/Desktop/Test/NewsSummarization/data/generate_data/{domain.name}/Comment")
        
        for comment in comments.glob("*.json"):
            with open(comment, "r", encoding="utf-8") as f:
                json_obj = json.load(f)
            
            sql_obj = PNComment(
                pn_evaluation_id = json_obj["pnEvaluationId"],
                positive_comment= json_obj["positiveComment"],
                negative_comment=json_obj["negativeComment"],
                news_id=json_obj["newsId"]
                
            )
            table.append(str(tuple(sql_obj)))

        with open(Path(f"/Users/minji/Desktop/Test/NewsSummarization/data/SQL/04_PNComment/{domain.name}.txt"),
                "w",
                encoding="utf-8") as f:
            
            f.write(",\n".join(table))
    return

def transfer_summary(root: Path):
    table = []
    for domain in root.iterdir():
        if (domain.name == ".DS_Store") : continue

        for type in domain.iterdir():
            if (type.name == "Comment") or (type.name == ".DS_Store") : continue
            
            for summary in type.glob("*.json"):
                with open(summary, "r", encoding="utf-8") as f:
                    json_data = json.load(f)
                    
                sql_obj = Summary(
                    summary_id = json_data["summary_id"],
                    level=json_data["level"],
                    summary_content=json_data["summaryContent"],
                    news_id=json_data["newsId"]
                )
                
                table.append(str(tuple(sql_obj)))
        
        with open(domain.parent / f"{domain.name}.txt", "w", encoding="utf-8") as f:
            f.write(",\n".join(table))
    return

In [238]:
root_word= Path("/Users/minji/Desktop/Test/NewsSummarization/data/SQL/03_Word") ; c_cnt = 1


for domain in root_word.glob("*.txt"):
    new_table = [] ; 
    
    with open(domain, "r", encoding="utf-8") as f:
        table = f.readlines()
        
    for record in table:
        record = list(eval(record.strip(",\n")))
        record.insert(0,c_cnt) ; c_cnt += 1
        
        new_table.append(str(tuple(record)))
        
    with open(domain, "w", encoding="utf-8") as f:
        f.write(",\n".join(new_table))
        
        
    

## News 테이블 uuid 주입

In [78]:
import uuid

root = Path("/Users/minji/Desktop/Test/NewsSummarization/data/SQL/01_News")

for domain in root.glob("*.txt"):
    matrix = []
    
    with open(domain, "r", encoding="utf-8") as f:
        table = f.readlines()
        
    for row in table:
        record = list(eval(row.strip(",\n"), {"UUID": uuid.UUID}))
        record[1] = str(uuid.uuid4())#uuid 생성
        matrix.append(str(tuple(record)))
        
    with open(domain, "w", encoding="utf-8") as f:
        f.write(",\n".join(matrix))
        
        

# Word생성

In [ ]:

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

using_model = "gpt-4o"


def create_word(level: str, data):
    def decorator(func: Callable):
        def wrapper(*args, **kwargs):
            word_prompt =  f"""
            당신은 텍스트 전문 AI 어시스턴트입니다. 당신은 주어진 텍스트에 대하여 사용자의 어휘력 수준에 따라서 특정 단어 사전적 정의와 예문을 제시하여야합니다.
            우리는 {data["content"]}에서 3개의 단어를 추출합니다. 아래의 규칙을 따라주세요.

            1. 사용자의 어휘 수준은 "{level}"입니다.
            2. "Low" 수준일 경우: 일반인이 어려워할 고급 어휘를 중심으로 선정하세요.
            3. "High" 수준일 경우: 전문 용어나 도메인 지식을 포함한 단어를 중심으로 선정하세요.
            3.  출력 형식은 {word_template}형식에 맞게 json으로 제공합니다.
            4. newsId에는 {data["news_id"]} 값을 입력합니다.
            """
            return func(word_prompt)
        return wrapper
    return decorator


def generate(word_prompt):
    response = client.chat.completions.create (
        model = using_model,
        messages = [
            {"role": "user",
            "content" : word_prompt
            }
        ]
    )
    
    return response.choices[0].message.content.strip()
    

In [103]:
root = Path("/Users/minji/Desktop/Test/NewsSummarization/data/output")
save_path = Path("/Users/minji/Desktop/Test/NewsSummarization/data/generate_data")
cnt = 1
for domain in root.iterdir():
    if domain.name == ".DS_Store" or domain.name == "process_data.ipynb": continue
    
    for data in domain.iterdir():
        if data.name == ".DS_Store" : continue
        
        with open(data, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        for level in ["Low", "High"]:
            decorate_func = create_word(level,data)(generate)
            result = decorate_func()
            save_dir = save_path / domain.name / "Word"
            if not save_dir.exists() : save_dir.mkdir(parents=True, exist_ok=True)
            file_name = f"OutputJson_word_{level}_{cnt}.txt"
            
            with open(save_dir / file_name , "w", encoding="utf-8") as f:
                f.write(result)
            
        cnt += 1
        
        
        
    

# 정규식으로 코드블록 마커 지우기

In [ ]:
import re

for domain in Path("/Users/minji/Desktop/Test/NewsSummarization/data/generate_data").iterdir():
    if not domain.is_dir(): continue
    path = Path(f"/Users/minji/Desktop/Test/NewsSummarization/data/generate_data/{domain.name}/Word")
    
    for file in path.iterdir():
        if file.name == ".DS_Store": continue
        
        with open(file, "r", encoding="utf-8") as f:
            data = f.read()
        
        cleaned = re.sub(r'^```(json)?\s*$', '', data, flags=re.MULTILINE)
        # 문자열을 JSON으로 변환
        json_obj = json.loads(cleaned)
        
        #확장자 변환
        with open(file.with_suffix(".json"), "w", encoding="utf-8") as f:
            json.dump(json_obj, f, ensure_ascii=False, indent=4)        

In [106]:
for domain in Path("/Users/minji/Desktop/Test/NewsSummarization/data/generate_data").iterdir():
    if not domain.is_dir(): continue
    path = Path(f"/Users/minji/Desktop/Test/NewsSummarization/data/generate_data/{domain.name}/Word")
    
    for file in path.iterdir():
        if file.name.startswith == ".": continue
        
        if file.suffix == ".txt" or "txt" in file.name : file.unlink()